In [1]:
import pandas as pd
import numpy as np

In [2]:
# Chargement des DataSets
fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')
heart_df = pd.read_csv('../../../Data/Classification/Indicators of Heart Disease/heart_2022_no_nans_clean.csv')
retail_df = pd.read_csv('../../../Data/Clustering/retail Data/retail_data_clean.csv')

/var/folders/qc/m20c7_k95j52q7xcg6lwyd7w0000gn/T/ipykernel_15749/3500090452.py:2: DtypeWarning: Columns (25,30) have mixed types. Specify dtype option on import or set low_memory=False.
  fps_df = pd.read_csv('../../../Data/Regression/fps in video games/fps-in-video-games_clean.csv')


In [3]:
def consistent_representation(df, pollution_percentages, max_variations=3, target_column=None):
    df_dict = {}

    for pollution_percentage in pollution_percentages:
        df_polluted = df.copy()

        # Sélectionner les colonnes catégoriques, en excluant la colonne cible
        if target_column:
            categorical_columns = df_polluted.select_dtypes(include=['object', 'category']).columns
            categorical_columns = [col for col in categorical_columns if col != target_column]
        else:
            categorical_columns = df_polluted.select_dtypes(include=['object', 'category']).columns

        for col in categorical_columns:
            # Obtenir les valeurs uniques dans la colonne
            unique_values = df_polluted[col].unique()

            # Créer des variations pour chaque valeur unique
            variations_dict = {}
            for value in unique_values:
                variations = [value]  # Commencez avec la valeur originale
                for i in range(1, max_variations):
                    variations.append(f"{value}_var{i}")
                variations_dict[value] = variations

            # Appliquer des variations sur un sous-ensemble des lignes
            mask = np.random.rand(len(df_polluted)) < pollution_percentage
            df_polluted.loc[mask, col] = df_polluted.loc[mask, col].apply(
                lambda x: np.random.choice(variations_dict[x]) if x in variations_dict else x
            )

        # Ajouter le DataFrame pollué au dictionnaire avec le pourcentage de pollution comme clé
        df_dict[f"{int(pollution_percentage * 100)}%"] = df_polluted

    return df_dict

In [4]:
# Liste des pourcentages de valeurs manquantes à introduire
missing_percentages = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]

# Génération des DataFrames avec valeurs manquantes
fps_df_with_consistent = consistent_representation(fps_df, missing_percentages, 3,'FPS')
heart_df_with_consistent = consistent_representation(heart_df, missing_percentages, 3,'HadHeartAttack')
retail_df_with_consistent = consistent_representation(retail_df, missing_percentages, 3)

In [5]:
# Enregistrement des DataFrames générés
for key, value in fps_df_with_consistent.items():
    value.to_csv(f"../../../Data/Regression/fps in video games/Consistent Representation/fps_{key}.csv", index=False)

for key, value in heart_df_with_consistent.items():
    value.to_csv(f"../../../Data/Classification/Indicators of Heart Disease/Consistent Representation/heart_{key}.csv", index=False)

for key, value in retail_df_with_consistent.items():
    value.to_csv(f"../../../Data/Clustering/retail Data/Consistent Representation/retail_{key}.csv", index=False)